# Composites

In [34]:
import glob, os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.plot
import datetime
from pathlib import Path
from IPython.display import display

print('All libraries successfully imported!')

All libraries successfully imported!


## Set directory

In [35]:
computer_path = '/export/miro/ndeffense/LBRAT2104/'
grp_letter    = 'O'

data_path = f'{computer_path}data/'                     # Directory with data shared by the assistant
work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'  # Directory for all work files

# Image timeserie directory
ts_path = f'{work_path}NBR/'
raster_file_list = sorted(glob.glob(f'{ts_path}*.tif'))

# Output directory
composite_path = f'{work_path}COMPOSITE/'

Path(composite_path).mkdir(parents=True, exist_ok=True)

## Set parameters

Compositing method :
- mean
- median
- maximum
- minimum

In [36]:
method = 'median'

### Date
The date must be in each image filename. `begin_date` is the first position of the date

E.g. : "SITE_3_**2**0181108_NDVI.tif" --> **begin_date = 8**

In [37]:
format     = '%Y%m%d' # format : YYYYMMDD
begin_date = 8

year  = '2019'
start = f'{year}0101'
end   = f'{year}1231'

## Set filenames

In [38]:
composite_tif = f'{composite_path}composite_{method}_{year}_ND.tif'

## Get dataframe with all images
- date of acquisition
- file path

In [39]:
dict_list = []

for im in raster_file_list:
        
        print(im)
            
        date_str = os.path.basename(im)[begin_date-1:begin_date-1+8]

        date_im = datetime.datetime.strptime(date_str, format).date()

        dict_list.append({'date': date_im,
                          'im_path': im})

im_date_path_df = pd.DataFrame.from_dict(dict_list).sort_values('date')

display(im_date_path_df)

/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190405T000241_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190410T000249_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190415T000241_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190420T000249_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190425T000241_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190430T000249_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190505T000241_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190510T000249_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190515T000241_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190520T000249_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20190525T000241_NBR.tif
/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/NBR/T56HKH_20200305T000239_NBR.tif
/export/miro/ndeffense/LBRAT

,date,im_path
0,2019-04-05,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
1,2019-04-10,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
2,2019-04-15,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
3,2019-04-20,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
4,2019-04-25,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
5,2019-04-30,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
6,2019-05-05,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
7,2019-05-10,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
8,2019-05-15,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
9,2019-05-20,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...


### Select images inside a period

In [40]:
start_date = datetime.datetime.strptime(start, format).date()
end_date   = datetime.datetime.strptime(end, format).date()

mask = (im_date_path_df['date'] > start_date) & (im_date_path_df['date'] <= end_date)

im_date_path_df = im_date_path_df.loc[mask]

display(im_date_path_df)

,date,im_path
0,2019-04-05,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
1,2019-04-10,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
2,2019-04-15,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
3,2019-04-20,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
4,2019-04-25,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
5,2019-04-30,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
6,2019-05-05,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
7,2019-05-10,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
8,2019-05-15,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...
9,2019-05-20,/export/miro/ndeffense/LBRAT2104/GROUP_O/WORK/...


In [41]:
list_src_arr = []

for i, row in im_date_path_df.iterrows():
    im_file    = row['im_path']
    src = rasterio.open(im_file, "r")
    im_arr = src.read(1)
    list_src_arr.append(im_arr)
    src.close()
    
print(f'Shape of features : {im_arr.shape}')
print(f'Number of features : {len(list_src_arr)}')

Shape of features : (7831, 7171)
Number of features : 11


In [42]:
im_arr_stack = np.dstack(list_src_arr)

print(im_arr_stack.shape)
print(f'There are {im_arr_stack.shape[2]} features')
print(f'The features type is : {im_arr_stack.dtype}')

(7831, 7171, 11)
There are 11 features
The features type is : float64


In [43]:
if method == 'median':
    compo_arr = np.nanmedian(im_arr_stack, axis=2)

elif method == 'mean':
    compo_arr = np.nanmean(im_arr_stack, axis=2)

elif method == 'std':
    compo_arr = np.nanstd(im_arr_stack, axis=2)

elif method == 'minimum':
    compo_arr = np.nanmin(im_arr_stack, axis=2)

elif method == 'maximum':
    compo_arr = np.nanmax(im_arr_stack, axis=2)

else:
    print(f'Method not available !')

print(compo_arr)

/usr/lib64/python3.6/site-packages/numpy/lib/nanfunctions.py:1115: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


[[0.13754227 0.14192521 0.15314707 ... 0.68004169 0.64980545 0.67152009]
 [0.13102889 0.11373701 0.12652456 ... 0.72244898 0.69684639 0.70433953]
 [0.11227482 0.1061478  0.13364833 ... 0.70749543 0.66449318 0.68837593]
 ...
 [0.4954955  0.60790514 0.63305424 ... 0.6718894  0.68949045 0.69077107]
 [0.50523013 0.60673162 0.64029909 ... 0.66680797 0.69589258 0.66601753]
 [0.40720497 0.56546598 0.6709597  ... 0.68605578 0.69743988 0.67355372]]


In [44]:
img_temp_tif = raster_file_list[0]

with rasterio.open(img_temp_tif) as src:
    profile = src.profile

print(profile)

with rasterio.open(composite_tif, "w", **profile) as dst:
    dst.write(compo_arr, 1)

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': -10000.0, 'width': 7171, 'height': 7831, 'count': 1, 'crs': CRS({'init': 'epsg:32756'}), 'transform': Affine(10.0, 0.0, 202830.0,
       0.0, -10.0, 6269560.0), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
